In [10]:
import codecs
import glob
import math
import os
from decimal import Decimal
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline


data_dir = "data/CCSD_NC/Z/*.cif"
result_dir = data_dir.replace("*.cif", "")
paths = [p for p in glob.glob(data_dir)]

space_group_IT_number_key = "space_group_IT_number"
space_group_name_HM_alt_key = "space_group_name_H-M_alt"
cell_length_a_key = "cell_length_a"
cell_length_b_key = "cell_length_b"
cell_length_c_key = "cell_length_c"
cell_angle_alpha_key = "cell_angle_alpha"                
cell_angle_beta_key = "cell_angle_beta"             
cell_angle_gamma_key = "cell_angle_gamma"              
cell_volume_key = "cell_volume"                  
cell_formula_units_Z_key = "cell_formula_units_Z"           
cell_measurement_temperature_key = "cell_measurement_temperature"    
exptl_crystal_description_key = "exptl_crystal_description"    
exptl_crystal_colour_key = "exptl_crystal_colour"       
refine_ls_R_factor_gt_key = "refine_ls_R_factor_gt" 
refine_ls_wR_factor_ref_key = "refine_ls_wR_factor_ref"

# 追加
exptl_crystal_density_diffrn_key = "exptl_crystal_density_diffrn" #密度 g/cm-3 Dcalcd
diffrn_reflns_av_R_equivalents_key = "diffrn_reflns_av_R_equivalents" #Rint
reflns_number_total_key = "reflns_number_total" #反射数
diffrn_reflns_theta_max_key = "diffrn_reflns_theta_max" #2倍したものを2θmaxとして抜き出す
refine_ls_restrained_S_all_key = "refine_ls_restrained_S_all" #GoFall
refine_diff_density_max_key = "refine_diff_density_max"
refine_diff_density_min_key = "refine_diff_density_min"

# 8/2追加

cell_measurement_theta_max_key = "cell_measurement_theta_max"

# μ(Mo-Kα)
exptl_absorpt_coefficient_mu_key = "exptl_absorpt_coefficient_mu"

# refrections measured 
diffrn_reflns_number_key = "diffrn_reflns_number"

# refrections unique 
reflns_number_gt_key = "reflns_number_gt"



# ここから次の改行までの対称操作の数でZを割った値がZ'
space_group_symop_operation_xyz_key = "_space_group_symop_operation_xyz"



df_output = pd.DataFrame()
index_array = ["Space group IT number", 
               "Space group", 
               "a / Å", 
               "b / Å", 
               "c / Å", 
               "α / °",
               "β / °", 
               "γ / °", 
               "V / Å\u00B3", 
               "Dcalc / g cm-3",
               "Z", 
               "Z'",
               "T / K", 
               "2θ",
               "R", 
               "Rw", 
               "Rint",
               "GoFall",
               "reflns_number_total",
               "refine_diff_density_max",
               "refine_diff_density_min",
               "crystal description", 
               "cell_measurement_theta_max",
               "exptl_absorpt_coefficient_mu",
               "diffrn_reflns_number",
               "reflns_number_gt",
               "crystal colour"]

for path in paths:
    # file_nameから角度の情報を取る
    file_name = path.split('/')[-1]

    with codecs.open(path, "r", "UTF-8", "ignore") as file:
        df = pd.read_table(file)
        df = df.rename(columns={df.columns[0]: 'Col_1'})
#         keyを含む行を探す
        space_group_IT_number = df[df['Col_1'].str.contains(space_group_IT_number_key, case=False)]['Col_1'].iloc[0].split()[-1]
        space_group_name_HM_alt = df[df['Col_1'].str.contains(space_group_name_HM_alt_key, case=False)]['Col_1'].iloc[0].split("'")[-2]
        cell_length_a = df[df['Col_1'].str.contains(cell_length_a_key, case=False)]['Col_1'].iloc[0].split()[-1]
        cell_length_b = df[df['Col_1'].str.contains(cell_length_b_key, case=False)]['Col_1'].iloc[0].split()[-1]
        cell_length_c = df[df['Col_1'].str.contains(cell_length_c_key, case=False)]['Col_1'].iloc[0].split()[-1]
        cell_angle_alpha = df[df['Col_1'].str.contains(cell_angle_alpha_key, case=False)]['Col_1'].iloc[0].split()[-1]
        cell_angle_beta = df[df['Col_1'].str.contains(cell_angle_beta_key, case=False)]['Col_1'].iloc[0].split()[-1]
        cell_angle_gamma = df[df['Col_1'].str.contains(cell_angle_gamma_key, case=False)]['Col_1'].iloc[0].split()[-1]
        cell_volume = df[df['Col_1'].str.contains(cell_volume_key, case=False)]['Col_1'].iloc[0].split()[-1]
        cell_formula_units_Z = df[df['Col_1'].str.contains(cell_formula_units_Z_key, case=False)]['Col_1'].iloc[0].split()[-1]
        cell_measurement_temperature = df[df['Col_1'].str.contains(cell_measurement_temperature_key, case=False)]['Col_1'].iloc[0].split()[-1]
        refine_ls_R_factor_gt = df[df['Col_1'].str.contains(refine_ls_R_factor_gt_key, case=False)]['Col_1'].iloc[0].split()[-1]
        refine_ls_wR_factor_ref = df[df['Col_1'].str.contains(refine_ls_wR_factor_ref_key, case=False)]['Col_1'].iloc[0].split()[-1]
        exptl_crystal_description = df[df['Col_1'].str.contains(exptl_crystal_description_key, case=False)]['Col_1'].iloc[0].split()[-1]
        exptl_crystal_colour = df[df['Col_1'].str.contains(exptl_crystal_colour_key, case=False)]['Col_1'].iloc[0].split()[-1]
        exptl_crystal_density_diffrn = df[df['Col_1'].str.contains(exptl_crystal_density_diffrn_key, case=False)]['Col_1'].iloc[0].split()[-1]
        diffrn_reflns_av_R_equivalents = df[df['Col_1'].str.contains(diffrn_reflns_av_R_equivalents_key, case=False)]['Col_1'].iloc[0].split()[-1]
        refine_ls_restrained_S_all = df[df['Col_1'].str.contains(refine_ls_restrained_S_all_key, case=False)]['Col_1'].iloc[0].split()[-1]
        reflns_number_total = df[df['Col_1'].str.contains(reflns_number_total_key, case=False)]['Col_1'].iloc[0].split()[-1]
        diffrn_reflns_theta_max = float(df[df['Col_1'].str.contains(diffrn_reflns_theta_max_key, case=False)]['Col_1'].iloc[0].split()[-1]) * 2
        
        refine_diff_density_max = df[df['Col_1'].str.contains(refine_diff_density_max_key, case=False)]['Col_1'].iloc[0].split()[-1]
        refine_diff_density_min = df[df['Col_1'].str.contains(refine_diff_density_min_key, case=False)]['Col_1'].iloc[0].split()[-1]
#         8/2
        cell_measurement_theta_max = df[df['Col_1'].str.contains(cell_measurement_theta_max_key, case=False)]['Col_1'].iloc[0].split()[-1]
        exptl_absorpt_coefficient_mu = df[df['Col_1'].str.contains(exptl_absorpt_coefficient_mu_key, case=False)]['Col_1'].iloc[0].split()[-1]
        diffrn_reflns_number = df[df['Col_1'].str.contains(diffrn_reflns_number_key, case=False)]['Col_1'].iloc[0].split()[-1]
        reflns_number_gt = df[df['Col_1'].str.contains(reflns_number_gt_key, case=False)]['Col_1'].iloc[0].split()[-1]
        
#         Z'の計算
        cell_length_a_index = df[df['Col_1'].str.contains(cell_length_a_key, case=False)]['Col_1'].index[0]
        space_group_symop_operation_xyz_index = df[df['Col_1'].str.contains(space_group_symop_operation_xyz_key, case=False)]['Col_1'].index[0]
        Z_prime = round(int(cell_formula_units_Z) / (cell_length_a_index - space_group_symop_operation_xyz_index - 1))
        
        
#         CSVデータで取り出し
        data_array = [space_group_IT_number, 
                      space_group_name_HM_alt, 
                      cell_length_a, 
                      cell_length_b, 
                      cell_length_c, 
                      cell_angle_alpha, 
                      cell_angle_beta, 
                      cell_angle_gamma, 
                      cell_volume, 
                      exptl_crystal_density_diffrn,
                      cell_formula_units_Z, 
                      Z_prime,
                      cell_measurement_temperature, 
                      diffrn_reflns_theta_max,
                      refine_ls_R_factor_gt, 
                      refine_ls_wR_factor_ref, 
                      diffrn_reflns_av_R_equivalents,
                      refine_ls_restrained_S_all,
                      reflns_number_total,
                      refine_diff_density_max,
                      refine_diff_density_min,
                      exptl_crystal_description, 
                      cell_measurement_theta_max,
                      exptl_absorpt_coefficient_mu,
                      diffrn_reflns_number,
                      reflns_number_gt,
                      exptl_crystal_colour]
    
        df_result = pd.DataFrame(data_array, index=index_array, columns=[file_name])

        df_output = pd.concat([df_output, df_result], axis=1)
        
df_output.to_csv(result_dir + "output.csv", encoding="utf-8")
        

In [11]:
df_output

,NpOMe(R)3OH_UV.cif,NpOMeC10en_UV.cif,NpOMeC4_UV.cif,NpOMeC5_UV.cif,NpOMeC6_UV.cif,NpOMeC7_UV.cif,NpOMeC8_UV.cif,NpOMeC9_UV.cif
Space group IT number,4,2,14,14,2,14,2,14
Space group,P 21,P -1,P 21/c,P 21/c,P -1,P 21/n,P -1,P 21/c
a / Å,8.4175(2),8.2652(3),14.2702(4),12.8448(2),8.1676(3),12.7940(5),8.1650(8),17.7834(3)
b / Å,12.4046(3),12.5751(4),12.4711(3),8.39410(10),13.0477(6),8.3129(3),12.9604(8),13.4511(2)
c / Å,13.5768(3),20.1316(6),8.4604(2),30.3410(6),16.1976(7),33.0603(12),18.5410(14),8.19980(10)
α / °,90,75.242(3),90,90,79.395(4),90,101.732(6),90
β / °,101.0445(18),89.690(3),103.863(3),97.1402(15),81.210(3),98.150(3),100.891(8),91.7899(16)
γ / °,90,89.860(3),90,90,89.368(3),90,91.375(7),90
V / Å³,1391.37(6),2023.33(12),1461.80(7),3246.01(9),1676.48(12),3480.6(2),1882.5(3),1960.49(5)
Dcalc / g cm-3,1.367,1.203,1.292,1.221,1.238,1.246,1.201,1.201


In [6]:
df

,Col_1
0,_audit_creation_method 'SHELXL-2018/1'
1,_shelx_SHELXL_version_number '2018/1'
2,_chemical_name_systematic ?
3,_chemical_name_common ?
4,_chemical_melting_point ?
...,...
18121,TREF
18122,HKLF 4
18123,END
18124,;
